In [1]:
import os
import math

In [2]:
import numpy as np
import mxnet as mx
import json
import matplotlib.pyplot as plt
import cv2

In [3]:
import random
from shapely.geometry import LineString

In [4]:
dat_path = '/mnt/6B133E147DED759E/tmp/ylb/zp'
mask_path = '/mnt/6B133E147DED759E/tmp/ylb/zp_mask'

In [5]:
with open(dat_path + '/bj2.json') as f:
    db = json.load(f)

In [6]:
imgfiles = [i for i in os.listdir(dat_path) if i.endswith('.jpg')]
maskfiles = [i for i in os.listdir(mask_path) if i.endswith('.png')]

In [7]:
db_dict = {}

In [8]:
for i,j in enumerate(db):
    fn = os.path.split(j['filename'])[1]
    db_dict[fn] = i

In [9]:
img_cache = []
mask_cache = []

In [10]:
for i in imgfiles:
#     f = open(dat_path + '/' + i, 'rb')
#     img = f.read()
#     img = np.frombuffer(img, dtype=np.uint8)
#     f.close()
    
    img = cv2.imread(dat_path + '/' + i)   
    mask = cv2.imread(mask_path + '/' + i[:-4] + '.png', cv2.IMREAD_GRAYSCALE)
    img_cache.append(img)
    mask_cache.append(mask)

In [11]:
for i,fn in enumerate(imgfiles):
    idx = db_dict[fn]
    info = db[idx]
    anno = info['annotations']
    center_x = anno[0]['x']
    center_y = anno[0]['y']
    mark0_x = anno[1]['x']
    mark0_y = anno[1]['y']
    mark1_x = anno[2]['x']
    mark1_y = anno[2]['y']
    mark2_x = anno[3]['x']
    mark2_y = anno[3]['y']
    mark3_x = anno[4]['x']
    mark3_y = anno[4]['y']
    point_x = anno[5]['x']
    point_y = anno[5]['y']
    
    x0 = info['rect']['x0']
    y0 = info['rect']['y0']
    x1 = info['rect']['x1']
    y1 = info['rect']['y1']
    ylb_type = info['type']

    img = img_cache[i] #cv2.imdecode(img_cache[i], cv2.CV_LOAD_IMAGE_COLOR)
    mask = mask_cache[i]
    
    
    dst_w = 224.
    dst_h = 224.
    obj_w = x1 - x0
    obj_h = y1 - y0
    xc = (x0 + x1) / 2.
    yc = (y0 + y1) / 2.
    scale = dst_w / max(obj_w, obj_h)
    
    scale_jitter = random.uniform(0.75, 1.2)
    scale *= scale_jitter
    dst_w_jitter = random.uniform(-20,20)
    dst_h_jitter = random.uniform(-20,20)
    theta = random.uniform(0, 2*np.pi)
    
    
    T0 = np.array([[1,0,-xc],[0,1,-yc],[0,0,1]])
    S = np.array([[scale,0,0],[0, scale,0],[0,0,1]])
    R = np.array([[np.cos(theta), np.sin(theta), 0], [-np.sin(theta), np.cos(theta), 0],[0,0,1]])
    T1 = np.array([[1,0,dst_w/2. + dst_w_jitter],[0,1,dst_h/2. + dst_h_jitter],[0,0,1]])
    M = np.dot(S, T0)
    M = np.dot(R, M)
    M = np.dot(T1, M)
    M_warp = M[0:2,:]
    
    dst_img = cv2.warpAffine(img, M_warp, dsize=(int(dst_w), int(dst_h)))
    
    points = np.array([[center_x, center_y, 1],[mark0_x, mark0_y,1],
                       [mark1_x, mark1_y,1], [mark2_x,mark2_y,1],[mark3_x,mark3_y,1]])
                       
    points2 = np.dot(points, M.transpose())
    
    #tmp = points2[:,2]
    #tmp = tmp[:, np.newaxis]
    #print(M, tmp)
    #points2 = points2[:,:2] / tmp
    #print(points2)
    
#     cv2.circle(dst_img, (int(points2[0,0]), int(points2[0,1])), 4, (0,255,0),3)
#     cv2.circle(dst_img, (int(points2[1,0]), int(points2[1,1])), 4, (0,255,0),3)
#     cv2.circle(dst_img, (int(points2[2,0]), int(points2[2,1])), 4, (0,255,0),3)
#     cv2.circle(dst_img, (int(points2[3,0]), int(points2[3,1])), 4, (0,255,0),3)
#     cv2.circle(dst_img, (int(points2[4,0]), int(points2[4,1])), 4, (0,255,0),3)

    cv2.imwrite('./ylbres/' + fn, dst_img)
    
#     img = cv2.imread(dat_path + '/' + fn)
    
#     mask = cv2.imread(mask_path + '/' + fn[:-4] + '.png', cv2.IMREAD_GRAYSCALE)

#     contours, hierarchy = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#     contour = contours[0][:,0,:]
    
#     xmin = np.min(contour[:,0])
#     ymin = np.min(contour[:,1])
#     xmax = np.max(contour[:,0])
#     ymax = np.max(contour[:,1])

#     info[u'rect'] = {u'x0': int(xmin), u'y0': int(ymin), u'x1': int(xmax), u'y1': int(ymax)}


#     a = LineString([(center_x, center_y), (point_x, point_y)])
#     red_line = LineString([(mark0_x,mark0_y), (mark1_x, mark1_y)])
#     green_line = LineString([(mark1_x, mark1_y), (mark2_x,mark2_y)])
#     yellow_line = LineString([(mark2_x,mark2_y), (mark3_x,mark3_y)])

#     ylb_type = None
    
#     if a.intersects(red_line):
#         cv2.putText(img, 'red', (10,44), 0, 2.4, (255,0,255))
#         ylb_type = 0
#     elif a.intersects(green_line):
#         cv2.putText(img, 'green', (10,44), 0, 2.4, (255,0,255))
#         ylb_type = 1
#     elif a.intersects(yellow_line):
#         cv2.putText(img, 'yellow', (10,44), 0, 2.4, (255,0,255))
#         ylb_type = 2
#     else:
#         cv2.putText(img, 'red', (10,44), 0, 2.4, (255,0,255))
#         ylb_type = 0

#     info[u'type'] = ylb_type

    
    # ellipse = cv2.fitEllipse(contour)
    
#     px = mark0_x - center_x
#     py = mark0_y - center_y
#     angle = math.atan2(py, px)
#     angle = angle * 180. / math.pi
    
#     cv2.putText(img, str(angle), (10,14), 0, 0.4, (255,0,255))
    
#     cv2.drawContours(img,contours,-1,(0,0,255),3)
#     cv2.ellipse(img,ellipse,(0,0,255),2)

#     cv2.putText(img, str(ylb_type), (10,44), 0, 2.4, (255,0,255))

#     cv2.circle(img, (int(center_x), int(center_y)), 4, (0,255,0),3)
#     cv2.circle(img, (int(mark0_x), int(mark0_y)), 4, (0,255,0),3)
#     cv2.circle(img, (int(mark1_x), int(mark1_y)), 4, (0,255,0),3)
#     cv2.circle(img, (int(mark2_x), int(mark2_y)), 4, (0,255,0),3)
#     cv2.circle(img, (int(mark3_x), int(mark3_y)), 4, (0,255,0),3)
#     cv2.circle(img, (int(point_x), int(point_y)), 4, (0,255,0),3)
#     cv2.line(img, (int(center_x), int(center_y)), (int(mark0_x), int(mark0_y)), (0,255,255),3)
#     cv2.line(img, (int(center_x), int(center_y)), (int(point_x), int(point_y)), (0,0,255),3)
    
#     cv2.line(img, (int(mark0_x), int(mark0_y)), (int(mark1_x), int(mark1_y)), (0,0,255),3)
#     cv2.line(img, (int(mark1_x), int(mark1_y)), (int(mark2_x), int(mark2_y)), (0,0,255),3)
#     cv2.line(img, (int(mark2_x), int(mark2_y)), (int(mark3_x), int(mark3_y)), (0,0,255),3)

#     cv2.rectangle(img, (int(x0), int(y0)), (int(x1), int(y1)), (0,255,0),3)

#     cv2.imwrite('./ylbres/' + fn, img)

In [12]:
with open(dat_path + '/bj2.json', 'w') as f:
    json.dump(db, f)